<a href="https://colab.research.google.com/github/Rit-ctrl/Pytorch-repo/blob/main/Pytorch-step-by-step/Chap_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chapter 1 - Linear regression

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn
from torchviz import make_dot

ModuleNotFoundError: ignored